In [1]:
import os
import cv2
import glob
import numpy as np

# =========================
# Config (tune as needed)
# =========================
image_glob = "Killaloe/*.jpg"   # <-- change if needed
RATIO = 0.80
RANSAC_REPROJ_THRESH = 4.0
MIN_MATCH_COUNT = 8
BACK_WINDOW = 20
MAX_FEATURES = 8000
DOWNSCALE_FOR_FEATURES = 1.0

# Output scale (rendering only)
OUTPUT_DOWNSCALE = 0.35

# Transform sanity thresholds
SCALE_MIN, SCALE_MAX = 0.7, 1.3
MAX_SHEAR_COS = 0.2
MAX_SHIFT_FACTOR = 1.2

# Feather & exposure options
FEATHER_BLENDING = True
FEATHER_GAMMA = 1.0    # 1.0 = linear; try 0.7 (softer) or 1.5 (sharper edges)
EXPOSURE_GAIN = True
GAIN_TARGET = 128.0    # target mean gray for warped area
GAIN_MIN, GAIN_MAX = 0.6, 1.6  # safety caps to avoid overcorrection

# Transform cache
TRANSFORM_FILE = "mosaic_transforms.npz"
USE_SAVED_TRANSFORMS = True

# =========================
# Helpers
# =========================
def load_images(pattern):
    files = sorted(glob.glob(pattern))
    if not files:
        raise IOError("No images found. Check the glob/path.")
    imgs = [cv2.imread(f) for f in files]
    ok = [(f, im) for f, im in zip(files, imgs) if im is not None]
    if not ok:
        raise IOError("Images could not be read. Check formats/paths.")
    files = [f for f, _ in ok]
    imgs  = [im for _, im in ok]
    print(f"Loaded {len(imgs)} images.")
    return files, imgs

def small(im, scale):
    if scale == 1.0:
        return im
    h, w = im.shape[:2]
    return cv2.resize(im, (int(w*scale), int(h*scale)))

def prep_gray_for_features(img):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    g = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    g = clahe.apply(g)
    g = cv2.GaussianBlur(g, (3,3), 0)
    return g

def scale_pts(pts, scale):
    return pts * (1.0 / scale)

def transform_ok(H_2x3, img_shape):
    h, w = img_shape[:2]
    A = H_2x3[:, :2]
    t = H_2x3[:, 2]
    s1 = np.linalg.norm(A[:, 0]) + 1e-12
    s2 = np.linalg.norm(A[:, 1]) + 1e-12
    scale = 0.5*(s1+s2)
    if not (SCALE_MIN <= scale <= SCALE_MAX):
        return False
    cosang = abs(np.dot(A[:,0]/s1, A[:,1]/s2))
    if cosang > MAX_SHEAR_COS:
        return False
    diag = np.hypot(h, w)
    if np.linalg.norm(t) > MAX_SHIFT_FACTOR*diag:
        return False
    return True

def to_homography_from_affine(A2x3):
    H = np.eye(3, dtype=np.float32)
    H[:2, :] = A2x3.astype(np.float32)
    return H

# =========================
# Compute transforms (or load cache)
# =========================
if USE_SAVED_TRANSFORMS and os.path.exists(TRANSFORM_FILE):
    data = np.load(TRANSFORM_FILE, allow_pickle=True)
    H_to_base = list(data["H_to_base"])
    sizes = list(data["sizes"])
    files = list(data["files"])
    T = data["T"]
    W = int(data["W"])
    Hh = int(data["Hh"])
    print(f"Loaded transforms from {TRANSFORM_FILE}")
else:
    files, images = load_images(image_glob)

    # SIFT + FLANN
    try:
        sift = cv2.SIFT_create(nfeatures=MAX_FEATURES)
    except AttributeError:
        sift = cv2.xfeatures2d.SIFT_create(nfeatures=MAX_FEATURES)
    flann = cv2.FlannBasedMatcher(dict(algorithm=1, trees=5), dict(checks=64))

    kps, descs, sizes = [], [], []
    for im in images:
        im_small = small(im, DOWNSCALE_FOR_FEATURES)
        gs = prep_gray_for_features(im_small)
        kp, des = sift.detectAndCompute(gs, None)
        kps.append(kp)
        descs.append(des)
        sizes.append(im.shape[:2])

    def match_des(i, j):
        if descs[i] is None or descs[j] is None:
            return []
        matches = flann.knnMatch(descs[i], descs[j], k=2)
        good = []
        for m, n in matches:
            if m.distance < RATIO * n.distance:
                good.append(m)
        return good

    def find_H_to_anchor(i, j):
        good = match_des(i, j)
        print(f"Image {i} - {j}: {len(good)} good matches")
        if len(good) < 4:
            return None

        src = np.float32([kps[i][m.queryIdx].pt for m in good]).reshape(-1,1,2)
        dst = np.float32([kps[j][m.trainIdx].pt for m in good]).reshape(-1,1,2)
        src = scale_pts(src, DOWNSCALE_FOR_FEATURES)
        dst = scale_pts(dst, DOWNSCALE_FOR_FEATURES)

        # 1) Similarity
        A_sim, _ = cv2.estimateAffinePartial2D(src, dst, method=cv2.RANSAC,
                                               ransacReprojThreshold=RANSAC_REPROJ_THRESH)
        if A_sim is not None and transform_ok(A_sim, sizes[i]):
            return to_homography_from_affine(A_sim)

        # 2) Affine
        A_aff, _ = cv2.estimateAffine2D(src, dst, method=cv2.RANSAC,
                                        ransacReprojThreshold=RANSAC_REPROJ_THRESH)
        if A_aff is not None and transform_ok(A_aff, sizes[i]):
            return to_homography_from_affine(A_aff)

        # 3) Homography (last resort, clamp projective)
        H, mask = cv2.findHomography(src, dst, cv2.RANSAC, RANSAC_REPROJ_THRESH)
        if H is not None and H.shape == (3,3):
            if abs(H[2,0]) < 1e-4 and abs(H[2,1]) < 1e-4:
                A = H[:2,:2]
                s1 = np.linalg.norm(A[:,0]) + 1e-12
                s2 = np.linalg.norm(A[:,1]) + 1e-12
                scale = 0.5*(s1+s2)
                if SCALE_MIN <= scale <= SCALE_MAX:
                    return H.astype(np.float32)
        return None

    H_to_base = [np.eye(3, dtype=np.float32)]
    for i in range(1, len(images)):
        H_acc = None
        for win in (BACK_WINDOW, 2*BACK_WINDOW):
            lookbacks = range(1, min(win, i) + 1)
            for k in lookbacks:
                j = i - k
                if H_to_base[j] is None:
                    continue
                H_ij = find_H_to_anchor(i, j)
                if H_ij is not None:
                    H_acc = (H_to_base[j] @ H_ij).astype(np.float32)
                    break
            if H_acc is not None:
                break

        if H_acc is None:
            print(f"Unable to place image {i} (no robust anchor found up to {2*BACK_WINDOW}). Skipping.")
            H_to_base.append(None)
        else:
            H_to_base.append(H_acc)

    if not any(H is not None for H in H_to_base):
        raise RuntimeError("No images could be placed.")

    # Compute mosaic bounds
    corners = []
    for idx, H in enumerate(H_to_base):
        if H is None:
            continue
        h, w = sizes[idx]
        pts = np.float32([[0,0],[w,0],[w,h],[0,h]]).reshape(-1,1,2)
        warped = cv2.perspectiveTransform(pts, H)
        corners.append(warped)

    all_c = np.vstack(corners)
    x_min, y_min = all_c.min(axis=0).ravel()
    x_max, y_max = all_c.max(axis=0).ravel()

    W = int(np.ceil(x_max - x_min))
    Hh = int(np.ceil(y_max - y_min))
    tx = -int(np.floor(x_min)) if x_min < 0 else 0
    ty = -int(np.floor(y_min)) if y_min < 0 else 0
    T = np.array([[1,0,tx],[0,1,ty],[0,0,1]], dtype=np.float32)

    print(f"Mosaic canvas (full-res coords): {W} x {Hh}")
    np.savez(TRANSFORM_FILE,
             H_to_base=np.array(H_to_base, dtype=object),
             sizes=np.array(sizes, dtype=object),
             files=np.array(files, dtype=object),
             T=T, W=W, Hh=Hh)
    print(f"Saved transforms to {TRANSFORM_FILE}")

# =========================
# Render with feathered blending (+ optional exposure gain)
# =========================
S = np.array([[OUTPUT_DOWNSCALE, 0, 0],
              [0, OUTPUT_DOWNSCALE, 0],
              [0, 0, 1]], dtype=np.float32)
W_out = max(1, int(W * OUTPUT_DOWNSCALE))
H_out = max(1, int(Hh * OUTPUT_DOWNSCALE))
print(f"Rendering output canvas: {W_out} x {H_out} (scale={OUTPUT_DOWNSCALE})")

accum = np.zeros((H_out, W_out, 3), np.float32)
weight = np.zeros((H_out, W_out), np.float32)

for idx, H in enumerate(H_to_base):
    if H is None:
        continue

    # Load image on demand
    img = cv2.imread(files[idx])
    if img is None:
        print(f"Warning: could not read {files[idx]} at render time; skipping.")
        continue

    H_off = (S @ T @ H).astype(np.float32)

    warped = cv2.warpPerspective(
        img, H_off, (W_out, H_out),
        flags=cv2.INTER_LINEAR,
        borderMode=cv2.BORDER_CONSTANT, borderValue=0
    )

    # Binary mask of warped region
    mask = cv2.warpPerspective(
        np.ones((sizes[idx][0], sizes[idx][1]), np.uint8), H_off, (W_out, H_out),
        flags=cv2.INTER_NEAREST, borderValue=0
    )

    # Optional: exposure gain to reduce brightness seams
    if EXPOSURE_GAIN and np.any(mask):
        gray = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
        m = gray[mask > 0].mean() if (mask > 0).any() else 0.0
        gain = np.clip((GAIN_TARGET / max(m, 1.0)), GAIN_MIN, GAIN_MAX)
        warped = np.clip(warped.astype(np.float32) * gain, 0, 255).astype(np.uint8)

    # Feather blending weight
    if FEATHER_BLENDING:
        # Distance to seam (distance to nearest zero in mask)
        # mask must be non-zero where valid
        dt = cv2.distanceTransform((mask > 0).astype(np.uint8), cv2.DIST_L2, 3).astype(np.float32)
        if dt.max() > 0:
            dt /= (dt.max() + 1e-6)   # normalize 0..1
        w = dt ** FEATHER_GAMMA
    else:
        w = (mask > 0).astype(np.float32)

    accum += warped.astype(np.float32) * w[..., None]
    weight += w

weight[weight == 0] = 1e-6
mosaic = (accum / weight[..., None]).clip(0,255).astype(np.uint8)

out_name = "stitched_mosaic_feathered.jpg"
cv2.imwrite(out_name, mosaic)
print(f"Saved {out_name}")

Loaded transforms from mosaic_transforms.npz
Rendering output canvas: 18943 x 11552 (scale=0.35)
Saved stitched_mosaic_feathered.jpg
